In [31]:
import os
import pandas as pd
import numpy as np
from geopy import distance
from math import sqrt
from datetime import datetime
from joblib import dump, load

In [2]:
df = pd.read_csv("extract.csv")

In [3]:
df.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year
0,2014-02-19 07:22:00.00000074,12.50,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1,2014
1,2014-12-06 20:36:22.0000008,4.00,2014-12-06 20:36:22 UTC,-73.979815,40.751902,-73.979446,40.755481,1,2014
2,2014-02-19 16:03:00.000000160,5.50,2014-02-19 16:03:00 UTC,-73.976075,40.752422,-73.981082,40.759285,1,2014
3,2015-06-21 21:46:34.0000001,22.54,2015-06-21 21:46:34 UTC,-74.010483,40.717667,-73.985771,40.660366,1,2015
4,2015-01-11 03:15:38.0000001,58.00,2015-01-11 03:15:38 UTC,-73.983330,40.738720,-73.933197,40.847225,1,2015


In [4]:
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])


In [5]:
df["weekday"] = df["pickup_datetime"].dt.weekday_name
df["month"] = df["pickup_datetime"].dt.month_name()
df["week"] = df["pickup_datetime"].dt.week

In [6]:
df["hour"] = df["pickup_datetime"].dt.hour

In [13]:
def rush_hour(hour):
    if (hour > 16) & (hour < 20):
        return 1
    else :
        return 0

In [8]:
df["Rush_hour"] = df["hour"].apply(rush_hour, axis = 1)

In [15]:
df = df.drop(axis=0, index=190432)
df = df.drop(axis=0, index=284519)

KeyError: '[190432] not found in axis'

In [16]:
df["Distance"] = df.apply(lambda row: distance.distance((row["pickup_longitude"],row["pickup_latitude"]),
                                       (row["dropoff_longitude"],row["dropoff_latitude"])).km, axis = 1)

In [17]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,month,week,hour,Rush_hour,Distance
0,2014-02-19 07:22:00.00000074,12.50,2014-02-19 07:22:00+00:00,-73.986430,40.760465,-73.988990,40.737075,1,2014,Wednesday,February,8,7,0,0.775049
1,2014-12-06 20:36:22.0000008,4.00,2014-12-06 20:36:22+00:00,-73.979815,40.751902,-73.979446,40.755481,1,2014,Saturday,December,49,20,0,0.117733
2,2014-02-19 16:03:00.000000160,5.50,2014-02-19 16:03:00+00:00,-73.976075,40.752422,-73.981082,40.759285,1,2014,Wednesday,February,8,16,0,0.597511
3,2015-06-21 21:46:34.0000001,22.54,2015-06-21 21:46:34+00:00,-74.010483,40.717667,-73.985771,40.660366,1,2015,Sunday,June,25,21,0,3.273818
4,2015-01-11 03:15:38.0000001,58.00,2015-01-11 03:15:38+00:00,-73.983330,40.738720,-73.933197,40.847225,1,2015,Sunday,January,2,3,0,6.520476


In [18]:
df = df[df["Distance"]<1000]

def remove_outliers(df):
    return df.loc[(df["Distance"] >1 ) & (df["Distance"] < 100)]

In [19]:
remove_outliers(df)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,month,week,hour,Rush_hour,Distance
3,2015-06-21 21:46:34.0000001,22.54,2015-06-21 21:46:34+00:00,-74.010483,40.717667,-73.985771,40.660366,1,2015,Sunday,June,25,21,0,3.273818
4,2015-01-11 03:15:38.0000001,58.00,2015-01-11 03:15:38+00:00,-73.983330,40.738720,-73.933197,40.847225,1,2015,Sunday,January,2,3,0,6.520476
7,2014-05-01 09:12:00.000000198,7.00,2014-05-01 09:12:00+00:00,-73.966203,40.767500,-73.980915,40.774240,6,2014,Thursday,May,18,9,0,1.655071
9,2014-04-29 18:28:00.0000005,16.50,2014-04-29 18:28:00+00:00,-73.970323,40.750840,-73.998392,40.724862,6,2014,Tuesday,April,18,18,1,3.233346
10,2015-04-12 10:19:05.0000001,11.50,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6,2015,Sunday,April,15,10,0,2.918219
11,2015-05-12 23:51:42.0000007,5.00,2015-05-12 23:51:42+00:00,-73.954796,40.779335,-73.944931,40.780087,1,2015,Tuesday,May,20,23,0,1.101233
12,2014-12-08 16:00:01.0000001,16.50,2014-12-08 16:00:01+00:00,-73.982800,40.745287,-74.014250,40.702935,1,2014,Monday,December,50,16,0,3.744359
14,2014-03-21 11:00:29.0000003,8.00,2014-03-21 11:00:29+00:00,-73.950688,40.828531,-73.964101,40.808140,1,2014,Friday,March,12,11,0,1.623867
15,2014-10-02 13:39:56.0000004,12.00,2014-10-02 13:39:56+00:00,-73.995387,40.749482,-73.975789,40.748741,1,2014,Thursday,October,40,13,0,2.187415
16,2014-05-19 06:26:00.00000034,7.50,2014-05-19 06:26:00+00:00,-73.990815,40.730495,-73.978337,40.751085,1,2014,Monday,May,21,6,0,1.530309


In [24]:
df = pd.get_dummies(df, columns = ['weekday'], drop_first=True)

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = df[["Distance", "Rush_hour", "weekday_Monday", "weekday_Saturday", "weekday_Sunday", "weekday_Thursday", "weekday_Wednesday", "weekday_Tuesday"]].values
y = df["fare_amount"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
reg = LinearRegression().fit(X_train, y_train)

In [32]:
dump(reg, 'model.joblib') 

['model.joblib']

In [27]:
from sklearn.metrics import mean_squared_error
y_true = y_test
y_pred = reg.predict(X_test)

In [28]:
sqrt(mean_squared_error(y_true, y_pred))

6.82562125910368

In [29]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
sns.regplot(x="Distance", y="fare_amount", data=df);

In [ ]:
districts = {
    "Bronx" : (40.8517687,-73.9109737),
    "Queens" : (40.8517727,-73.9131624),
    "Staten Island": (40.5647149,-74.2168582),
    "Brooklyn" : (40.6453531,-74.0150372),
    "Manhattan" : (40.7591704,-74.0392707)}

In [ ]:
for key, value in districts.items():
    print(key, value)

In [ ]:
distance.distance()

## Night and Day

In [ ]:
df["hour"] = df[df["pickup_datetime"]<1000] Series.dt.hour

In [ ]:
df["pickup_datetime"].sort_values

In [30]:
ls

environment.yml        taxi-fare.ipynb
extract.csv            taxi_fare_extract.csv


In [34]:
df[["Distance", "Rush_hour", "weekday_Monday", "weekday_Saturday", "weekday_Sunday", "weekday_Thursday", "weekday_Wednesday", "weekday_Tuesday"]].iloc[2]

Distance             0.597511
Rush_hour            0.000000
weekday_Monday       0.000000
weekday_Saturday     0.000000
weekday_Sunday       0.000000
weekday_Thursday     0.000000
weekday_Wednesday    1.000000
weekday_Tuesday      0.000000
Name: 2, dtype: float64